Considerando a regionalidade do SESI, poderiamos combinar os dados do pysus com ibge atraves 
from pysus.online_data import IBGE
pop = IBGE.get_population(source="POP", year=2010)
pop
Ou simplesmente regionalizar o estudo, aqui se der tempo vou tentar ver o que tem maior ocorrencia por estado e plotar em barras empilhadas com numero de casos/pop

In [ ]:
#pip install datasus-fetcher

In [41]:
pip install pandas-dbc

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pandas-dbc (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for pandas-dbc


In [ ]:
import datasus_fetcher
import os
import pandas as pd
import dbc

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Exemplo de pré-processamento para a seleção de colunas
df_cluster = df_acgr_cru[['DT_NOTIFIC', 'SG_UF_NOT',   'ANO_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'ID_OCUPA_N', 
                           'SIT_TRAB', 'NUTEMPO', 'TPTEMPO', 'LOCAL_ACID', 'UF_EMP', 'MUN_EMP', 
                           'TERCEIRIZA', 'HORA_JOR', 'MIN_JOR', 'TIPO_ACID']]

# Convertendo colunas de data
df_cluster['DT_NOTIFIC'] = pd.to_datetime(df_cluster['DT_NOTIFIC'])
df_cluster['Ano_Mes'] = df_cluster['DT_NOTIFIC'].dt.to_period('M')

df_cluster['NU_IDADE_N'] = decodifica_idade_SINAN(df_cluster.NU_IDADE_N, 'Y')

# Convertendo as colunas numéricas para o tipo adequado
numeric_cols = ['ANO_NASC', 'NUTEMPO', 'TPTEMPO', 'HORA_JOR', 'MIN_JOR', 'TERCEIRIZA']

# Convertendo para numérico, forçando erros para NaN e tratando
df_cluster[numeric_cols] = df_cluster[numeric_cols].apply(pd.to_numeric, errors='coerce')
# Garantindo que as variáveis numéricas sejam tratadas como float
df_cluster[numeric_cols] = df_cluster[numeric_cols].astype(float)

# Tratando valores nulos nas variáveis numéricas
df_cluster[numeric_cols] = df_cluster[numeric_cols].fillna(df_cluster[numeric_cols].mean())

# Tratando valores nulos nas variáveis numéricas (preenchendo com a média)
df_cluster[numeric_cols] = df_cluster[numeric_cols].fillna(df_cluster[numeric_cols].mean())



# Codificando variáveis categóricas
categorical_cols = ['SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 
                    'CS_ESCOL_N', 'ID_OCUPA_N', 'SIT_TRAB', 'LOCAL_ACID', 'UF_EMP', 'MUN_EMP', 
                    'TIPO_ACID']

# Preenchendo os valores nulos nas colunas categóricas (com o valor 'Desconhecido', por exemplo)
df_cluster[categorical_cols] = df_cluster[categorical_cols].fillna('Desconhecido')


encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_data = encoder.fit_transform(df_cluster[categorical_cols])

# Transformando variáveis numéricas
numeric_cols = ['ANO_NASC', 'NU_IDADE_N', 'NUTEMPO', 'TPTEMPO', 'HORA_JOR', 'MIN_JOR', 'TERCEIRIZA']
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_cluster[numeric_cols])

# Convertendo a matriz esparsa para uma matriz densa
encoded_data_dense = encoded_data.toarray()

# Combinando os dados transformados
processed_data = np.concatenate([encoded_data_dense, scaled_data], axis=1)
